Load libraries

In [3]:
!pip install gensim --upgrade -q

     |████████████████████████████████| 24.1 MB 1.4 MB/s 


In [1]:
import gensim

In [2]:
gensim.__version__

'4.1.2'

In [3]:

import numpy as np
import pandas as pd
from sklearn.cluster import KMeans
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn import metrics
import tqdm
import nltk
import re
from operator import itemgetter

EPOCHS = 175
TOPICS = 20
CHUNK_SIZE = 1000
WORKERS = 7
EVAL_PERIOD = 10

In [4]:
nltk.download('punkt')
nltk.download('wordnet')
nltk.download('stopwords')

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Unzipping corpora/wordnet.zip.
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


True

Clean dataset

In [5]:
def preprocess_text(sentence):
    # Lowercase
    sentence = sentence.lower()
    
    # Remove all non-alphabets (punctuation, numbers, new-line characters and extra-spaces)
    sentence = re.sub(r'[^a-zA-Z]+', ' ', sentence)
    sentence = sentence.replace('\n', '')
    # sentence = re.sub('\s\s+', ' ', sentence)
    
    # Tokenize & remove stop-words
    word_list = nltk.word_tokenize(sentence)    
    stopwords_list = set(nltk.corpus.stopwords.words('english'))
    word_list = [word for word in word_list if word not in stopwords_list]
    
    # Remove very small words, length < 3, they don't contribute any useful information
    word_list = [word for word in word_list if len(word) > 3]
        
    # Stem & Lemmatize
    porter_stemmer = nltk.stem.PorterStemmer()
    lemmatizer = nltk.stem.WordNetLemmatizer()
    word_list = [porter_stemmer.stem(word) for word in word_list]
    word_list = [lemmatizer.lemmatize(word) for word in word_list]
    
    sentence = ' '.join(word_list)
    
    return sentence

Load data

In [6]:
from sklearn.datasets import fetch_20newsgroups
news_group = fetch_20newsgroups(subset='train')

news_group_data = news_group.data
news_group_target_names = news_group.target_names
news_group_target = news_group.target

# Creating a dataframe from the loaded data
news_df = pd.DataFrame({'news': news_group_data, 
                        'class': news_group_target})

news_df.reset_index(drop=True, inplace= True)

from sklearn.model_selection import train_test_split

train_df, test_df = train_test_split(news_df, test_size=0.2)

Pre-process

In [7]:
# Preprocess the news description
tqdm.tqdm.pandas()
train_df['news_tokenized'] = train_df['news'].progress_apply(lambda x: preprocess_text(str(x)))
test_df['news_tokenized'] = test_df['news'].progress_apply(lambda x: preprocess_text(str(x)))

100%|██████████| 2263/2263 [00:15<00:00, 150.51it/s]


TF-IDF

In [8]:
tf_idfvectorizer = TfidfVectorizer(strip_accents='unicode', stop_words='english', min_df=2)
x_train_tfidf = tf_idfvectorizer.fit_transform(train_df.news_tokenized)
x_test_tfidf = tf_idfvectorizer.transform(test_df.news_tokenized)

In [9]:
performance_metrics = pd.DataFrame(columns=['feature-extraction','clustering-algo', 'run#', 'state', 'AMI','ARI','NMI','time'])


K-Means

In [11]:
import time

In [12]:
for run, state in zip(range(1, 3, 1), range(2, 42, 2)):
    print('Run #', run)
    
    start = time.time()
    k_means = KMeans(n_clusters=TOPICS, init='k-means++', max_iter=EPOCHS, random_state=state)
    k_means.fit(x_train_tfidf)
    
    pred_labels = k_means.predict(x_test_tfidf)
        
    ami = metrics.adjusted_mutual_info_score(test_df['class'], pred_labels)
    ari = metrics.adjusted_rand_score(test_df['class'], pred_labels)
    nmi = metrics.normalized_mutual_info_score(test_df['class'], pred_labels)
    stop = time.time()
    
    performance_metrics = performance_metrics.append({'feature-extraction':'tf-idf', 'clustering-algo':'k-means', 'run#':run, 'state':state, 'AMI':ami, 
                                                      'ARI': ari, 'NMI':nmi, 'time':(stop-start)}, ignore_index=True)


Run # 1
Run # 2


NMF

In [13]:
train_documents = train_df['news_tokenized'].str.split()
dictionary = gensim.corpora.Dictionary(train_documents)
dictionary.filter_extremes(no_below=5, no_above=0.5, keep_n=20000)

test_documents = test_df['news_tokenized'].str.split()

tfidf_model = gensim.models.TfidfModel(dictionary=dictionary)

train_corpus = [dictionary.doc2bow(document) for document in train_documents]
test_corpus = [dictionary.doc2bow(document) for document in test_documents]

train_corpus_tfidf = list(tfidf_model[train_corpus])
test_corpus_tfidf = list(tfidf_model[test_corpus])

In [15]:
for run, state in zip(range(1, 3, 1), range(2, 42, 2)):
    print('Run #', run)
    
    start = time.time()
    gensim_nmf = gensim.models.Nmf(corpus=train_corpus_tfidf, num_topics=TOPICS, id2word=dictionary, chunksize=CHUNK_SIZE, passes=EPOCHS, eval_every=EVAL_PERIOD, minimum_probability=0, 
                                   random_state=state, kappa=1)
    
    pred_labels = []
    for test_doc in test_corpus_tfidf:
        pred_label = max(gensim_nmf[test_doc], key=itemgetter(1))[0]
        pred_labels.append(pred_label)
        
    ami = metrics.adjusted_mutual_info_score(test_df['class'], pred_labels)
    ari = metrics.adjusted_rand_score(test_df['class'], pred_labels)
    nmi = metrics.normalized_mutual_info_score(test_df['class'], pred_labels)
    stop = time.time()
        
    performance_metrics = performance_metrics.append({'feature-extraction':'tf-idf', 'clustering-algo':'NMF', 'run#':run, 'state':state, 'AMI':ami, 
                                                      'ARI': ari, 'NMI':nmi, 'time':(stop-start)}, ignore_index=True)


Run # 1
Run # 2


LDA

In [16]:
for run, state in zip(range(1, 2, 1), range(2, 42, 2)):
    print('Run #', run)
    
    start = time.time()
    gensim_lda = gensim.models.ldamulticore.LdaMulticore(corpus=train_corpus_tfidf, num_topics=TOPICS, id2word=dictionary, chunksize=CHUNK_SIZE, workers=WORKERS, passes=EPOCHS, 
                                                     eval_every = EVAL_PERIOD, per_word_topics=True, random_state=state)
    
    pred_labels = []
    for test_doc in test_corpus_tfidf:
        pred_label = max(gensim_lda[test_doc][0], key=itemgetter(1))[0]
        pred_labels.append(pred_label)
        
    ami = metrics.adjusted_mutual_info_score(test_df['class'], pred_labels)
    ari = metrics.adjusted_rand_score(test_df['class'], pred_labels)
    nmi = metrics.normalized_mutual_info_score(test_df['class'], pred_labels)
    stop = time.time()
        
    performance_metrics = performance_metrics.append({'feature-extraction':'tf-idf', 'clustering-algo':'LDA', 'run#':run, 'state':state, 'AMI':ami, 
                                                      'ARI': ari, 'NMI':nmi, 'time':(stop-start)}, ignore_index=True)


Run # 1


LDA Visualization

In [23]:
!pip install pyLDAvis

     |████████████████████████████████| 1.7 MB 5.1 MB/s 
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Installing backend dependencies ... done
    Preparing wheel metadata ... done
  Created wheel for pyLDAvis: filename=pyLDAvis-3.3.1-py2.py3-none-any.whl size=136898 sha256=a906b17e89b9d08c6e0c8d35995f9a77cb4b8ccca520e598b31d4db4b67e03f3
  Stored in directory: /root/.cache/pip/wheels/c9/21/f6/17bcf2667e8a68532ba2fbf6d5c72fdf4c7f7d9abfa4852d2f
Successfully built pyLDAvis


In [26]:
import pyLDAvis
#Creating Topic Distance Visualization 
import pyLDAvis.gensim_models as gensimvis
pyLDAvis.enable_notebook()

# feed the LDA model into the pyLDAvis instance
lda_viz = gensimvis.prepare(gensim_lda, test_corpus_tfidf, dictionary )

/usr/local/lib/python3.7/dist-packages/pyLDAvis/_prepare.py:247: FutureWarning: In a future version of pandas all arguments of DataFrame.drop except for the argument 'labels' will be keyword-only
  by='saliency', ascending=False).head(R).drop('saliency', 1)


In [28]:
lda_viz

PreparedData(topic_coordinates=              x         y  topics  cluster       Freq
topic                                                
13    -0.342008  0.162329       1        1  64.876653
16    -0.231633 -0.239291       2        1   6.311031
19     0.056027 -0.023159       3        1   2.564689
0      0.045705 -0.027556       4        1   2.377068
11     0.046563 -0.054666       5        1   2.104549
8      0.038778  0.003678       6        1   1.846397
4      0.037859 -0.000308       7        1   1.786413
18     0.034968  0.009302       8        1   1.658928
15     0.029291  0.029984       9        1   1.565275
9      0.026571  0.018169      10        1   1.534138
5      0.017290 -0.010595      11        1   1.497224
2      0.032867  0.009754      12        1   1.485467
3      0.032997  0.011896      13        1   1.484621
14     0.026519  0.000847      14        1   1.425713
12     0.029860  0.024989      15        1   1.347347
7      0.035942  0.020838      16        1   1.338018
10     0.018020  0.006385      17        1   1.276414
1      0.020508  0.022359      18        1   1.267593
17     0.025976  0.015697      19        1   1.174284
6      0.017900  0.019345      20        1   1.078178, topic_info=            Term       Freq      Total Category  logprob  loglift
43        window  21.000000  21.000000  Default  30.0000  30.0000
450         card  15.000000  15.000000  Default  29.0000  29.0000
666         chip  11.000000  11.000000  Default  28.0000  28.0000
148        drive  16.000000  16.000000  Default  27.0000  27.0000
1533        file  16.000000  16.000000  Default  26.0000  26.0000
...          ...        ...        ...      ...      ...      ...
4806       vacat   0.325038   0.835450  Topic20  -6.1645   3.5859
1329       digex   0.457340   4.762980  Topic20  -5.8230   2.1867
2285      helmet   0.356055   2.676835  Topic20  -6.0734   2.5126
2873        leaf   0.348461   2.694682  Topic20  -6.0949   2.4844
3068  informatik   0.338210   2.527359  Topic20  -6.1248   2.5186

[1005 rows x 6 columns], token_table=      Topic      Freq     Term
term                          
329       1  0.808103   access
329       2  0.161621   access
329       7  0.080810   access
7947      9  0.831788     acsu
882       3  0.792419  adaptec
...     ...       ...      ...
1183      5  1.202689     yoyo
9307      2  1.088516      zeo
3817     18  1.074983  zionism
3288      9  0.841720  zisfein
4522      7  0.750498   zoolog

[822 rows x 3 columns], R=30, lambda_step=0.01, plot_opts={'xlab': 'PC1', 'ylab': 'PC2'}, topic_order=[14, 17, 20, 1, 12, 9, 5, 19, 16, 10, 6, 3, 4, 15, 13, 8, 11, 2, 18, 7])

LSI

In [19]:
for run, state in zip(range(1, 3, 1), range(2, 42, 2)):
    print('Run #', run)
    
    start = time.time()
    gensim_lsi = gensim.models.LsiModel(corpus=train_corpus_tfidf, num_topics=TOPICS, id2word=dictionary, chunksize=CHUNK_SIZE)
    
    pred_labels = []
    for test_doc in test_corpus:
        pred_label = max(gensim_lsi[test_doc], key=itemgetter(1))[0]
        pred_labels.append(pred_label)
            
    ami = metrics.adjusted_mutual_info_score(test_df['class'], pred_labels)
    ari = metrics.adjusted_rand_score(test_df['class'], pred_labels)
    nmi = metrics.normalized_mutual_info_score(test_df['class'], pred_labels)
    stop = time.time()
      
    performance_metrics = performance_metrics.append({'feature-extraction':'Bag of Words', 'clustering-algo':'LSI', 'run#':run, 'state':state, 'AMI':ami, 
                                                      'ARI': ari, 'NMI':nmi, 'time':(stop-start)}, ignore_index=True)
    # break

Run # 1
Run # 2


HDP

In [20]:
for run, state in zip(range(1, 3, 1), range(2, 42, 2)):
    print('Run #', run)
    
    start = time.time()
    gensim_hdp = gensim.models.hdpmodel.HdpModel(corpus=train_corpus_tfidf, id2word=dictionary, chunksize=CHUNK_SIZE, random_state=state, kappa=1, alpha=0.01)
    pred_labels = []
    for test_doc in test_corpus:
        pred_label = max(gensim_lsi[test_doc], key=itemgetter(1))[0]
        pred_labels.append(pred_label)
    ami = metrics.adjusted_mutual_info_score(test_df['class'], pred_labels)
    ari = metrics.adjusted_rand_score(test_df['class'], pred_labels)
    nmi = metrics.normalized_mutual_info_score(test_df['class'], pred_labels)
    # topics = gensim_hdp.print_topics()
    stop = time.time()
      
    performance_metrics = performance_metrics.append({'feature-extraction':'Bag of Words', 'clustering-algo':'HDP', 'run#':run, 'state':state, 'AMI':ami, 
                                                      'ARI': ari, 'NMI':nmi,'time':(stop-start)}, ignore_index=True)

Run # 1
Run # 2


In [21]:
performance_metrics

feature-extraction clustering-algo run# state       AMI       ARI  \
0              tf-idf         k-means    1     2  0.432945  0.152264   
1              tf-idf         k-means    2     4  0.395782  0.113128   
2              tf-idf             NMF    1     2  0.467219  0.298985   
3              tf-idf             NMF    2     4  0.459631  0.290203   
4              tf-idf             LDA    1     2  0.025723  0.000662   
5        Bag of Words             LSI    1     2  0.043650  0.001257   
6        Bag of Words             LSI    2     4  0.043650  0.001257   
7        Bag of Words             HDP    1     2  0.043650  0.001257   
8        Bag of Words             LSI    1     2  0.089351  0.004145   
9        Bag of Words             LSI    2     4  0.079562  0.003290   
10       Bag of Words             HDP    1     2  0.079562  0.003290   
11       Bag of Words             HDP    2     4  0.079562  0.003290   

         NMI         time  
0   0.451518    24.145665  
1   0.415687    13.980870  
2   0.482907   142.367105  
3   0.475536   130.434171  
4   0.054395  1081.584181  
5   0.066841    11.928758  
6   0.066841    10.279693  
7   0.066841    78.735964  
8   0.120342    17.148435  
9   0.109348    15.261658  
10  0.109348    90.917667  
11  0.109348    86.955219

Save metrics

In [22]:
performance_metrics.to_csv('/content/drive/MyDrive/Big Data/Code/Metrics/evaluating_performance-tfidf.csv', index=False)